In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
import os
import statsmodels.api as sm

os.chdir("C:/Users/sungi/Documents/Merkle")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

def Get_team_records(model, teamdata, teamname):
    if teamname not in teamdata.index.values: # team does not have records
        #print('{} team has no previous records'.format(teamname))
        prob_teamWin = np.array([[0.0,0.0]]) 
    else: # avg on team records
        teamrecord = pd.DataFrame(teamdata.loc[teamname].values).T
        teamrecord.columns = teamdata.loc[teamname].index
        prob_teamWin = model.predict_proba(teamrecord)

    return np.reshape(prob_teamWin, 2)
  

def Predicted_Winner(model, teamdata, teamname_A, teamname_B, prob_arr = False):
   A_record = Get_team_records(model,avg_teams,teamname_A)
   B_record = Get_team_records(model,avg_teams,teamname_B)
   #Winteam = ''
   if A_record[1] > B_record[1]: Winteam = teamname_A
   elif A_record[1] < B_record[1]: Winteam = teamname_B
   else: Winteam = 'Both'
   
   if prob_arr == True :
       print()
       print('["%s" VS "%s"]' %(teamname_A, teamname_B))
       print('*Win Probabilities: %s(%.4f) / %s(%.4f)' %(teamname_A,A_record[1],teamname_B,B_record[1]))
       if Winteam == 'Both': print('Two teams might tie')
       else: print('*Predicted Result: "%s" might win the game.' %Winteam)
   
   return Winteam, A_record, B_record #if prob_arr == True else Winteam

def Predicted_Eval_Table(model, avg_teams, gameidTeam_test):
    pred_df = pd.DataFrame(columns=["predicted_win", "prob_Team_A", "prob_Team_B", "prob_diff", "err"])
    for index, row in gameidTeam_test.iterrows():
        teamWin, team_A, team_B = Predicted_Winner(model, avg_teams, row[1], row[2], prob_arr = False)
        
        # count error 
        if row[1] == teamWin: err = 0
        else: err = 1
        
        records = [teamWin, team_A[1], team_B[1], team_A[1]-team_B[1], err]
        records = pd.DataFrame([records], columns=["predicted_win", "prob_Team_A", "prob_Team_B", "prob_diff", "err"])
        pred_df = pred_df.append(records, ignore_index=True)
        
    gameidTeam_test.reset_index(drop=True, inplace=True)
    pred_df.reset_index(drop=True, inplace=True)
    results_tab = pd.concat([gameidTeam_test, pred_df], axis=1)
    totalerr = results_tab.err.sum()/results_tab.shape[0]
    print('Error rate : %.4f' %totalerr )
    return results_tab

def Review_diff_Error (pred_gameidTeam_test, result_table, prob_diff_range, title=''):    
    print('[Probability difference of {} cases (percentage of total err)]'.format(title))
    totalerr = pred_gameidTeam_test.err.sum()/pred_gameidTeam_test.shape[0]
    print('Error rate : %.4f' %totalerr )    
    for p in prob_diff_range:
        less_ = result_table[abs(result_table.prob_diff) < p].shape[0]
        print('prob difference is less than %.2f : %.d (%.4f)' %(p, less_,less_/result_table.shape[0]))

def Testing_model (model, avg_teams, gameidTeam_test):
    pred_gameidTeam_test = Predicted_Eval_Table(model, avg_teams, gameidTeam_test)
    
    pred_gameidTeam_test.head(10)
    
    # check the pattern on missclassified cases
    correct_Table = pred_gameidTeam_test[pred_gameidTeam_test.err == 0.0]
    incorrect_Table = pred_gameidTeam_test[pred_gameidTeam_test.err == 1.0]
    
    prob_diff_range = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
    
    Review_diff_Error(pred_gameidTeam_test, incorrect_Table, prob_diff_range,title='incorrect')
    Review_diff_Error(pred_gameidTeam_test, correct_Table, prob_diff_range,title='correct')
    
    correct_Table.prob_Team_A.mean()
    correct_Table.prob_Team_B.mean()
    incorrect_Table.prob_Team_A.mean()
    incorrect_Table.prob_Team_B.mean()
    return pred_gameidTeam_test

In [3]:
# input the original dataset
PlayerStats_2017 = pd.read_csv('2017PlayerStats Test.csv', sep=',')
TeamStats_2017 = pd.read_csv('2017TeamStats Test.csv', sep=',')
PlayerStats_2018 = pd.read_csv('2018PlayerStats Final.csv', sep=',')
TeamStats_2018 = pd.read_csv('2018TeamStats Final.csv', sep=',')

PlayerStats_2017.name = 'PlayerStats_2017'
TeamStats_2017.name = 'TeamStats_2017'
PlayerStats_2018.name = 'PlayerStats_2018'
TeamStats_2018.name = 'TeamStats_2018'

In [4]:
datalist = [PlayerStats_2017,TeamStats_2017,PlayerStats_2018,TeamStats_2018]

In [5]:
# check which columns have missing values
for i in datalist:
    print()
    print('Dataset Name: %s' %i.name)
    print(i.apply(lambda x: x.count(), axis=0))


Dataset Name: PlayerStats_2017
Unnamed: 0    122224
player        122224
MP            122224
FG            122224
FGA           122224
FG%           108005
2P            122224
2PA           122224
2P%            98124
3P            122224
3PA           122224
3P%            74065
FT            122224
FTA           122224
FT%            65514
ORB           122224
DRB           122224
TRB           122224
AST           122224
STL           122224
BLK           122224
TOV           122224
PF            122224
PTS           122224
Team          122224
gameid        122224
dtype: int64

Dataset Name: TeamStats_2017
Unnamed: 0    11932
Team          11932
MP            11932
FG            11932
FGA           11932
FG%           11932
2P            11932
2PA           11932
2P%           11932
3P            11932
3PA           11932
3P%           11932
FT            11932
FTA           11932
FT%           11929
ORB           11932
DRB           11932
TRB           11932
AST           11932

In [6]:
# fill the missing values with '0'
for i in datalist:
    i.fillna(0 ,inplace = True)

In [7]:
# recheck the missing value
for i in datalist:
    print()
    print('Dataset Name: %s' %i.name)
    print(i.apply(lambda x: x.count(), axis=0))


Dataset Name: PlayerStats_2017
Unnamed: 0    122224
player        122224
MP            122224
FG            122224
FGA           122224
FG%           122224
2P            122224
2PA           122224
2P%           122224
3P            122224
3PA           122224
3P%           122224
FT            122224
FTA           122224
FT%           122224
ORB           122224
DRB           122224
TRB           122224
AST           122224
STL           122224
BLK           122224
TOV           122224
PF            122224
PTS           122224
Team          122224
gameid        122224
dtype: int64

Dataset Name: TeamStats_2017
Unnamed: 0    11932
Team          11932
MP            11932
FG            11932
FGA           11932
FG%           11932
2P            11932
2PA           11932
2P%           11932
3P            11932
3PA           11932
3P%           11932
FT            11932
FTA           11932
FT%           11932
ORB           11932
DRB           11932
TRB           11932
AST           11932

In [8]:
# check the counts
print('total # of players in 2017: {}'.format(PlayerStats_2017.player.nunique()))
print('total # of players in 2018: {}'.format(PlayerStats_2018.player.nunique()))

total # of players in 2017: 8529
total # of players in 2018: 8383


In [9]:
print('total # of teams in 2017: {}'.format(TeamStats_2017.Team.nunique()))
print('total # of teams in 2018: {}'.format(TeamStats_2018.Team.nunique()))

total # of teams in 2017: 640
total # of teams in 2018: 638


In [10]:
print('total # of games in 2017: {}'.format(TeamStats_2017.gameid.nunique()))
print('total # of games in 2018: {}'.format(TeamStats_2018.gameid.nunique()))

total # of games in 2017: 5966
total # of games in 2018: 5684


In [11]:
# Create new table for further analysis
Team2017_winrank = TeamStats_2017.groupby('Team')['Win?'].agg(['count','sum']).sort_values(by=['count','sum'],ascending=[False, False])
Team2017_winrank

,count,sum
Team,,
north-carolina,40,33
gonzaga,39,37
oregon,39,33
texas-christian,39,24
coastal-carolina,39,20
kentucky,38,32
michigan,38,26
xavier,38,24
wyoming,38,23


In [12]:
# create new table based on 'gameid' (win = 'Team_x')
gameid_2017 = TeamStats_2017[['gameid','Team','Win?']]
gameid_2017[gameid_2017['Win?']== 1]
gameidTeam_2017 = pd.merge(gameid_2017[gameid_2017['Win?']== 1], gameid_2017[gameid_2017['Win?']== 0], on='gameid')
gameidTeam_2017 = gameidTeam_2017.drop(['Win?_x', 'Win?_y'], axis=1)
gameidTeam_2017.columns = ['gameid','Team_A','Team_B']
gameidTeam_2017.head(10)

,gameid,Team_A,Team_B
0,111120161,abilene-christian,schreiner
1,111120162,air-force,mcpherson
2,111120163,alabama,coastal-carolina
3,111120164,alabama-birmingham,arkansas-pine-bluff
4,111120165,arizona-state,portland-state
5,111120166,arkansas,ipfw
6,111120167,auburn,north-florida
7,111120168,baylor,oral-roberts
8,111120169,binghamton,cornell
9,1111201610,nicholls-state,boston-college


In [13]:
gameidTeam_2017.gameid.nunique() # check the # of gameid

5966

In [14]:
grpteam_2017 = TeamStats_2017.groupby('Team')

In [15]:
# Data preprocessing - remove fields
# 1. opponents records (i.e. field name contains 'Opp')
# 2. high correlated fields (i.e. 'FT' and 'FTA')

# delete fields related Opponents (remain records related field 'Team')
Team_2017_noopp = TeamStats_2017.drop(TeamStats_2017.columns[24:45], axis=1).iloc[:,1:]
Team_2017_noopp.columns # 25 columns left 

Index(['Team', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', '3P', '3PA',
       '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS', 'gameid', 'Win?'],
      dtype='object')

In [16]:
# correlation - trim the columns that might have high correlation
# correlation (heatmap)
sns.heatmap(Team_2017_noopp.iloc[:,1:23].corr(), 
            xticklabels=Team_2017_noopp.iloc[:,1:23].columns.values,
            yticklabels=Team_2017_noopp.iloc[:,1:23].columns.values)

# correlation (Top 20 correlated fields)
print("Top Absolute Correlations")
print(get_top_abs_correlations(Team_2017_noopp.iloc[:,1:23], 20))

Top Absolute Correlations
FT   FTA    0.927158
FG   PTS    0.870319
DRB  TRB    0.837100
FG%  2P%    0.836360
FG   2P     0.803996
     FG%    0.763730
FG%  PTS    0.718232
3P   3P%    0.710296
     3PA    0.691810
2P   2PA    0.683738
FGA  2PA    0.671435
FG   AST    0.660559
     2P%    0.644915
ORB  TRB    0.638786
2P   2P%    0.635800
AST  PTS    0.629584
FG%  2P     0.603759
2P   PTS    0.598569
FG%  AST    0.566398
2P%  PTS    0.561615
dtype: float64


In [17]:
# remove fields (remain 15 field)
Team_2017_nocorr = Team_2017_noopp.drop(['FG', 'FGA', 'FG%', '2P', '2PA','3P', '3PA','FT', 'FTA', 'TRB'], axis=1)
Team_2017_nocorr.columns

Index(['Team', 'MP', '2P%', '3P%', 'FT%', 'ORB', 'DRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS', 'gameid', 'Win?'],
      dtype='object')

In [18]:
# recheck the correleation
print("Top Absolute Correlations")
print(get_top_abs_correlations(Team_2017_nocorr.iloc[:,1:23], 20))

Top Absolute Correlations
AST  PTS     0.629584
2P%  PTS     0.561615
PTS  Win?    0.502072
3P%  PTS     0.490151
     AST     0.444621
2P%  AST     0.439434
DRB  Win?    0.424439
2P%  Win?    0.404770
AST  Win?    0.398819
DRB  PTS     0.350329
3P%  Win?    0.333995
DRB  BLK     0.276289
     AST     0.266573
BLK  Win?    0.243367
PF   Win?    0.242007
TOV  PF      0.232252
STL  PTS     0.211292
FT%  PTS     0.209682
2P%  DRB     0.195604
3P%  ORB     0.185941
dtype: float64


In [19]:
Team_2017_nocorr.head(10) # check the new dataset

,Team,MP,2P%,3P%,FT%,ORB,DRB,AST,STL,BLK,TOV,PF,PTS,gameid,Win?
0,schreiner,200,0.469,0.381,0.455,5,22,18,6,2,14,14,59,111120161,0
1,abilene-christian,200,0.429,0.409,0.667,15,26,13,8,5,11,17,73,111120161,1
2,mcpherson,200,0.517,0.353,0.556,11,18,13,2,0,10,27,71,111120162,0
3,air-force,200,0.714,0.364,0.706,4,26,20,4,1,6,17,88,111120162,1
4,coastal-carolina,200,0.205,0.391,0.625,10,21,7,2,2,13,22,53,111120163,0
5,alabama,200,0.500,0.385,0.692,7,29,13,8,8,11,19,70,111120163,1
6,arkansas-pine-bluff,200,0.333,0.385,0.706,11,16,10,7,1,17,28,66,111120164,0
7,alabama-birmingham,200,0.595,0.188,0.767,11,30,19,10,9,15,19,86,111120164,1
8,portland-state,200,0.500,0.222,0.800,8,21,14,5,1,14,20,70,111120165,0
9,arizona-state,200,0.615,0.321,0.722,11,29,11,7,3,16,15,88,111120165,1


In [20]:
# Data analysis - Logistic Regression

# 1. model will predict probabilities of winning based on record of one team 
#   and show the significant variables in your Model
#   (Here, it will not consider opponent ability yet)
#   (Accuracy from this model means accuracy on 'how well it will predict Win(0,1)' only based on one team records.)
#   In other words, team that has high score would be likely to win(1).)
# 2. get probabilities of winning for each game (for further analysis)
# 3. average the field records on each team 
#   (We are averaging team records to get 'average ability of team'. 
#   It will represent 'how well the team did in 2017/2018')
# 4. By apply 'average records of team' on model, 
#   it will show the probability of winning based on average records of previous games)
# 5. When we get the probabilities of winning on two teams seperatly,
#   the one who has higher probability would win the game. 
# 6. The function will input the team names, 
#   and output the probabilities on each team and which team would win the game.

# creating testing and training set
x_train,x_test,y_train,y_test = train_test_split(Team_2017_nocorr.iloc[:,:14],Team_2017_nocorr.iloc[:,14],test_size=0.34)

In [21]:
# perform Logistic Regression (exclude 'Team', 'gameid' for LR)
lr_x_train = x_train.iloc[:,1:13]
lr_x_test = x_test.iloc[:,1:13]

In [22]:
lrclf = LogisticRegression()
model = lrclf.fit(lr_x_train,y_train)
prob_test = model.predict_proba(lr_x_test)

In [23]:
print ('Score (Accuracy) on Test set: ', lrclf.score(lr_x_test,y_test))

Score (Accuracy) on Test set:  0.84249445403


In [24]:
col_list = lr_x_test.columns
print('logit[win=1] = %.4f' %model.intercept_, end=' ')
for i in range(np.reshape(col_list.values, 12).shape[0]):
    print('+ (%.4f*%s)'% (np.reshape(model.coef_, 12)[i],np.reshape(col_list.values, 12)[i]), end=' ')

logit[win=1] = -5.6006 + (-0.0480*MP) + (8.1920*2P%) + (6.8472*3P%) + (1.9249*FT%) + (0.0840*ORB) + (0.2664*DRB) + (0.0017*AST) + (0.2512*STL) + (0.1360*BLK) + (-0.1381*TOV) + (-0.1490*PF) + (0.0421*PTS) 

In [25]:
#removing MP & FT%
Team_2017_nocorrn0 = Team_2017_nocorr.drop(['PTS'], axis=1)
Team_2017_nocorrn0.columns

Index(['Team', 'MP', '2P%', '3P%', 'FT%', 'ORB', 'DRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'gameid', 'Win?'],
      dtype='object')

In [26]:
# creating testing and training set
x_trainn0,x_testn0,y_trainn0,y_testn0 = train_test_split(Team_2017_nocorrn0.iloc[:,:13],Team_2017_nocorrn0.iloc[:,13],test_size=0.34)

In [27]:
# perform Logistic Regression (exclude 'Team', 'gameid' for LR)
lr_x_trainn0 = x_trainn0.iloc[:,1:12]
lr_x_testn0 = x_testn0.iloc[:,1:12]

In [28]:
lrclf = LogisticRegression()
model0 = lrclf.fit(lr_x_trainn0,y_trainn0)
prob_test = model0.predict_proba(lr_x_testn0)

In [29]:
print ('Score (Accuracy) on Test set: ', lrclf.score(lr_x_testn0,y_testn0))
#model0 = scf.Testing_svm(lrclf, lr_x_trainn0, y_trainn0, lr_x_testn0, y_testn0, n_sv = False, clf_title='Logistic Regression')

Score (Accuracy) on Test set:  0.845945279763


In [30]:
# print Logistic model (y = a + bx ) and check the significant variables (fields)
# Answer : (8.2942*2P%) + (7.2058*3P%) + (2.0198*FT%) in an order
col_list = lr_x_testn0.columns
print('logit[win=1] = %.4f' %model0.intercept_, end=' ')
for i in range(np.reshape(col_list.values, 11).shape[0]):
    print('+ (%.4f*%s)'% (np.reshape(model0.coef_, 11)[i],np.reshape(col_list.values, 11)[i]), end=' ')

logit[win=1] = -6.6799 + (-0.0417*MP) + (10.3005*2P%) + (8.9374*3P%) + (2.4784*FT%) + (0.1241*ORB) + (0.2638*DRB) + (0.0183*AST) + (0.2543*STL) + (0.1355*BLK) + (-0.1587*TOV) + (-0.1260*PF) 

In [31]:
# create dataset has average records of teams (avg_teams)
#Team_2017_nocorrn0.head(10)
grpteam = x_trainn0.groupby('Team')
avg_teams = grpteam.aggregate(np.mean).iloc[:,:11]
print('Total # of teams who have records: {}'.format(len(avg_teams.index.values)))

Total # of teams who have records: 568


In [32]:
# call team matches that is assigned as test set by 'gameid' 
gameidTeam_test = gameidTeam_2017.loc[gameidTeam_2017['gameid'].isin(x_testn0['gameid'].values)] 
#(x_testn0.gameid.nunique() = gameidTeam_test.shape[0]])  # rows should be the same with unique count of test set 'gameid'

In [33]:
# create table to evaluate model
pred_gameidTeam_test = Testing_model(model0, avg_teams, gameidTeam_test)
pred_gameidTeam_test.head(5)

Error rate : 0.3236
[Probability difference of incorrect cases (percentage of total err)]
Error rate : 0.3236
prob difference is less than 0.05 : 206 (0.1916)
prob difference is less than 0.10 : 392 (0.3647)
prob difference is less than 0.20 : 683 (0.6353)
prob difference is less than 0.30 : 889 (0.8270)
prob difference is less than 0.40 : 1004 (0.9340)
prob difference is less than 0.50 : 1058 (0.9842)
[Probability difference of correct cases (percentage of total err)]
Error rate : 0.3236
prob difference is less than 0.05 : 220 (0.0979)
prob difference is less than 0.10 : 449 (0.1998)
prob difference is less than 0.20 : 898 (0.3996)
prob difference is less than 0.30 : 1319 (0.5870)
prob difference is less than 0.40 : 1673 (0.7445)
prob difference is less than 0.50 : 1927 (0.8576)


,gameid,Team_A,Team_B,predicted_win,prob_Team_A,prob_Team_B,prob_diff,err
0,111120161,abilene-christian,schreiner,abilene-christian,0.371650,0.253281,0.118370,0
1,111120162,air-force,mcpherson,air-force,0.506398,0.036232,0.470166,0
2,111120163,alabama,coastal-carolina,coastal-carolina,0.362514,0.697214,-0.334700,1
3,111120164,alabama-birmingham,arkansas-pine-bluff,alabama-birmingham,0.654001,0.074599,0.579402,0
4,111120165,arizona-state,portland-state,portland-state,0.549296,0.642080,-0.092785,1


In [34]:
#entire 17 & 18 data
frames = [TeamStats_2017, TeamStats_2018]
BigTDF = pd.concat(frames)
BigTDF = BigTDF[['Team', 'MP', '2P%', '3P%', 'FT%', 'ORB', 'DRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'gameid', 'Win?']]
BigTDF.head(10)



,Team,MP,2P%,3P%,FT%,ORB,DRB,AST,STL,BLK,TOV,PF,gameid,Win?
0,schreiner,200,0.469,0.381,0.455,5,22,18,6,2,14,14,111120161,0
1,abilene-christian,200,0.429,0.409,0.667,15,26,13,8,5,11,17,111120161,1
2,mcpherson,200,0.517,0.353,0.556,11,18,13,2,0,10,27,111120162,0
3,air-force,200,0.714,0.364,0.706,4,26,20,4,1,6,17,111120162,1
4,coastal-carolina,200,0.205,0.391,0.625,10,21,7,2,2,13,22,111120163,0
5,alabama,200,0.5,0.385,0.692,7,29,13,8,8,11,19,111120163,1
6,arkansas-pine-bluff,200,0.333,0.385,0.706,11,16,10,7,1,17,28,111120164,0
7,alabama-birmingham,200,0.595,0.188,0.767,11,30,19,10,9,15,19,111120164,1
8,portland-state,200,0.5,0.222,0.8,8,21,14,5,1,14,20,111120165,0
9,arizona-state,200,0.615,0.321,0.722,11,29,11,7,3,16,15,111120165,1


In [35]:
#use for prediction
Predicted_Winner(model0, BigTDF, 'ucla', 'gonzaga', prob_arr = False)

('ucla', array([ 0.02773595,  0.97226405]), array([ 0.04219907,  0.95780093]))

In [36]:
#use for searching team names
BigTDF.loc[BigTDF['Team'].str.contains('mich')]

,Team,MP,2P%,3P%,FT%,ORB,DRB,AST,STL,BLK,TOV,PF,gameid,Win?
37,central-michigan,200,0.561,0.294,0.82,16,51,10,9,9,8,12,1111201619,1
151,michigan,200,0.464,0.375,0.885,10,17,16,9,4,10,21,1111201676,1
153,michigan-state,200,0.515,0.32,0.556,10,18,15,4,1,17,23,1111201677,0
224,eastern-michigan,250,0.5,0.238,0.826,12,25,12,8,5,11,30,11112016113,0
317,western-michigan,200,0.519,0.148,0.571,24,33,11,13,2,15,24,11112016159,1
441,michigan,200,0.513,0.333,0.889,10,29,10,4,7,9,15,1113201622,1
482,eastern-michigan,200,0.5,0.242,0.833,9,16,7,10,6,14,27,1113201643,0
588,western-michigan,200,0.404,0.2,0.722,11,27,5,3,1,15,16,1114201646,0
632,central-michigan,200,0.541,0.333,0.765,13,27,14,6,2,10,20,1114201668,1
681,eastern-michigan,200,0.478,0.438,0.722,12,32,11,11,2,17,18,1115201615,1
